# **Ensemble of Ensembles: Model Stacking**

### *Ensemble with different type of classifiers:*

*   Different type of classifiers (logistic regression,decision tree,random forest etc) fitted on the same training data.
*   Results are combined based on : majority voting (classification) and average (regression).

### *Ensemble with same type of classifiers:*
*   Bootstarp samples are drawn from training data.
*   with each bootstrap sample, model (individual model may be logistic regression,decision tree ) will be fitted.
* all the results are combined to create an ensemble.
* suitable for highly flexible model that is prone to overfitting.

**Combining Method:**
1. Majority Voting.
2. Method of application of meta-classifier on outcome (labels- 0/1).
3. Method of application of meta-classifier on probabilities result of each classifier.






In [131]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sklearn
sns.set_style('whitegrid')

In [132]:
from google.colab import files
upload_files = files.upload()

Saving IBM_HR_Employee_Attrition.csv to IBM_HR_Employee_Attrition (1).csv


In [133]:
!ls

IBM_HR_Employee_Attrition.csv  sample_data


In [134]:
df = pd.read_csv('IBM_HR_Employee_Attrition.csv')
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,Y,Yes,15,3,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,4,Female,56,3,1,Research Scientist,3,Married,2909,23159,1,Y,Yes,11,3,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468,16632,9,Y,No,12,3,4,80,1,6,3,3,2,2,2,2


In [135]:
# get all the unique values of column
c = ['Attrition','BusinessTravel','Department','EducationField','Gender','MaritalStatus','OverTime','JobRole']
for i in c:
    print(i,':',df[i].unique())

Attrition : ['Yes' 'No']
BusinessTravel : ['Travel_Rarely' 'Travel_Frequently' 'Non-Travel']
Department : ['Sales' 'Research & Development' 'Human Resources']
EducationField : ['Life Sciences' 'Other' 'Medical' 'Marketing' 'Technical Degree'
 'Human Resources']
Gender : ['Female' 'Male']
MaritalStatus : ['Single' 'Married' 'Divorced']
OverTime : ['Yes' 'No']
JobRole : ['Sales Executive' 'Research Scientist' 'Laboratory Technician'
 'Manufacturing Director' 'Healthcare Representative' 'Manager'
 'Sales Representative' 'Research Director' 'Human Resources']


In [136]:
df['Attrition'] = df['Attrition'].map({'Yes':0,'No':1})
df['BusinessTravel'] = df['BusinessTravel'].map({'Travel_Rarely':0,'Travel_Frequently':1,'Non-Travel':2})
df['Department'] = df['Department'].map({'Sales':0,'Research & Development':1,'Human Resources':2})
df['EducationField'] = df['EducationField'].map({'Life Sciences':0,'Other':1,'Medical':2,'Marketing':3,
                                                 'Technical Degree':4,'Human Resources':5})
df['Gender'] = df['Gender'].map({'Male':0,'Female':1})
df['MaritalStatus'] = df['MaritalStatus'].map({'Single':0,'Married':1,'Divorced':2})
df['OverTime'] = df['OverTime'].map({'Yes':0,'No':1})
df['JobRole'] = df['JobRole'].map({'Sales Executive':0,'Research Scientist':1,'Laboratory Technician':2,
                                   'Manufacturing Director':3,'Healthcare Representative':4,'Manager':5,
                                   'Sales Representative':6,'Research Director':7,'Human Resources':8})

In [137]:
df = df.drop(['EmployeeCount','EmployeeNumber','StandardHours','Over18'],axis = 1)

In [138]:
y = df['Attrition']
y.head()

0    0
1    1
2    0
3    1
4    1
Name: Attrition, dtype: int64

In [139]:
x = df.drop(['Attrition'],axis = 1)
x

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,0,1102,0,1,2,0,2,1,94,3,2,0,4,0,5993,19479,8,0,11,3,1,0,8,0,1,6,4,0,5
1,49,1,279,1,8,1,0,3,0,61,2,2,1,2,1,5130,24907,1,1,23,4,4,1,10,3,3,10,7,1,7
2,37,0,1373,1,2,2,1,4,0,92,2,1,2,3,0,2090,2396,6,0,15,3,2,0,7,3,3,0,0,0,0
3,33,1,1392,1,3,4,0,4,1,56,3,1,1,3,1,2909,23159,1,0,11,3,3,0,8,3,3,8,7,3,0
4,27,0,591,1,2,1,2,1,0,40,3,1,2,2,1,3468,16632,9,1,12,3,4,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,1,884,1,23,2,2,3,0,41,4,2,2,4,1,2571,12290,4,1,17,3,3,1,17,3,3,5,2,0,3
1466,39,0,613,1,6,1,2,4,0,42,2,3,4,1,1,9991,21457,4,1,15,3,1,1,9,5,3,7,7,1,7
1467,27,0,155,1,4,3,0,2,0,87,4,2,3,2,1,6142,5174,1,0,20,4,2,1,6,0,3,6,2,0,3
1468,49,1,1023,0,2,3,2,4,0,63,2,2,0,2,1,5390,13243,2,1,14,3,4,0,17,3,2,9,6,0,8


In [140]:
x.astype('float64')

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41.0,0.0,1102.0,0.0,1.0,2.0,0.0,2.0,1.0,94.0,3.0,2.0,0.0,4.0,0.0,5993.0,19479.0,8.0,0.0,11.0,3.0,1.0,0.0,8.0,0.0,1.0,6.0,4.0,0.0,5.0
1,49.0,1.0,279.0,1.0,8.0,1.0,0.0,3.0,0.0,61.0,2.0,2.0,1.0,2.0,1.0,5130.0,24907.0,1.0,1.0,23.0,4.0,4.0,1.0,10.0,3.0,3.0,10.0,7.0,1.0,7.0
2,37.0,0.0,1373.0,1.0,2.0,2.0,1.0,4.0,0.0,92.0,2.0,1.0,2.0,3.0,0.0,2090.0,2396.0,6.0,0.0,15.0,3.0,2.0,0.0,7.0,3.0,3.0,0.0,0.0,0.0,0.0
3,33.0,1.0,1392.0,1.0,3.0,4.0,0.0,4.0,1.0,56.0,3.0,1.0,1.0,3.0,1.0,2909.0,23159.0,1.0,0.0,11.0,3.0,3.0,0.0,8.0,3.0,3.0,8.0,7.0,3.0,0.0
4,27.0,0.0,591.0,1.0,2.0,1.0,2.0,1.0,0.0,40.0,3.0,1.0,2.0,2.0,1.0,3468.0,16632.0,9.0,1.0,12.0,3.0,4.0,1.0,6.0,3.0,3.0,2.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36.0,1.0,884.0,1.0,23.0,2.0,2.0,3.0,0.0,41.0,4.0,2.0,2.0,4.0,1.0,2571.0,12290.0,4.0,1.0,17.0,3.0,3.0,1.0,17.0,3.0,3.0,5.0,2.0,0.0,3.0
1466,39.0,0.0,613.0,1.0,6.0,1.0,2.0,4.0,0.0,42.0,2.0,3.0,4.0,1.0,1.0,9991.0,21457.0,4.0,1.0,15.0,3.0,1.0,1.0,9.0,5.0,3.0,7.0,7.0,1.0,7.0
1467,27.0,0.0,155.0,1.0,4.0,3.0,0.0,2.0,0.0,87.0,4.0,2.0,3.0,2.0,1.0,6142.0,5174.0,1.0,0.0,20.0,4.0,2.0,1.0,6.0,0.0,3.0,6.0,2.0,0.0,3.0
1468,49.0,1.0,1023.0,0.0,2.0,3.0,2.0,4.0,0.0,63.0,2.0,2.0,0.0,2.0,1.0,5390.0,13243.0,2.0,1.0,14.0,3.0,4.0,0.0,17.0,3.0,2.0,9.0,6.0,0.0,8.0


In [141]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,roc_auc_score

x_train,x_test,y_train,y_test = train_test_split(x,y)

In [142]:
def print_score(model,x_train,y_train,x_test,y_test,part):
  if part == 'train':
    y_pred = model.predict(x_train)
    print("Training Performance:\n\n")
    print("Accuracy:{0:.4f}\n".format(accuracy_score(y_train,y_pred)))
    print("Clasification Report:\n{}\n".format(classification_report(y_train,y_pred)))
    print("Confusion Matrix:\n{}\n".format(confusion_matrix(y_train,y_pred)))
    print("ROC_AUC_Score:{0:.4f}\n".format(roc_auc_score(y_train,y_pred)))
   
  elif part == 'test':
    y_pred = model.predict(x_test)
    print("Testing Performance:\n\n")
    print("Accuracy:{0:.4f}\n".format(accuracy_score(y_test,y_pred)))
    print("Clasification Report:\n{}\n".format(classification_report(y_test,y_pred)))
    print("Confusion Matrix:\n{}\n".format(confusion_matrix(y_test,y_pred)))
    print("ROC_AUC_Score:{0:.4f}\n".format(roc_auc_score(y_test,y_pred)))
    

### **Model 1: DecisionTreeClassifier**

In [143]:
clf = tree.DecisionTreeClassifier(random_state= 42)
clf.fit(x_train,y_train.ravel())
print_score(clf,x_train,y_train,x_test,y_test,part = 'train')
print("\n***************************************************************\n")
print_score(clf,x_train,y_train,x_test,y_test,part = 'test')

Training Performance:


Accuracy:1.0000

Clasification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       180
           1       1.00      1.00      1.00       922

    accuracy                           1.00      1102
   macro avg       1.00      1.00      1.00      1102
weighted avg       1.00      1.00      1.00      1102


Confusion Matrix:
[[180   0]
 [  0 922]]

ROC_AUC_Score:1.0000


***************************************************************

Testing Performance:


Accuracy:0.7908

Clasification Report:
              precision    recall  f1-score   support

           0       0.33      0.35      0.34        57
           1       0.88      0.87      0.88       311

    accuracy                           0.79       368
   macro avg       0.61      0.61      0.61       368
weighted avg       0.80      0.79      0.79       368


Confusion Matrix:
[[ 20  37]
 [ 40 271]]

ROC_AUC_Score:0.6111



### **Model 2: RandomForestClassifier**

In [144]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 100)
rf.fit(x_train,y_train.ravel())
print_score(rf,x_train,y_train,x_test,y_test,part = 'train')
print("\n***************************************************************\n")
print_score(rf,x_train,y_train,x_test,y_test,part = 'test')

Training Performance:


Accuracy:1.0000

Clasification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       180
           1       1.00      1.00      1.00       922

    accuracy                           1.00      1102
   macro avg       1.00      1.00      1.00      1102
weighted avg       1.00      1.00      1.00      1102


Confusion Matrix:
[[180   0]
 [  0 922]]

ROC_AUC_Score:1.0000


***************************************************************

Testing Performance:


Accuracy:0.8641

Clasification Report:
              precision    recall  f1-score   support

           0       0.73      0.19      0.31        57
           1       0.87      0.99      0.92       311

    accuracy                           0.86       368
   macro avg       0.80      0.59      0.62       368
weighted avg       0.85      0.86      0.83       368


Confusion Matrix:
[[ 11  46]
 [  4 307]]

ROC_AUC_Score:0.5901



In [145]:
clf.predict_proba(x_train)[1]

array([0., 1.])

In [146]:
en = pd.DataFrame()

In [147]:
en['clf_Tree'] = pd.DataFrame(clf.predict_proba(x_train))[1]
en['rf_Forest'] = pd.DataFrame(rf.predict_proba(x_train))[1]
col = en.columns
en = pd.concat([en,pd.DataFrame(y_train).reset_index(drop = True)],axis = 1)

In [148]:
en

,clf_Tree,rf_Forest,Attrition
0,1.0,0.98,1
1,1.0,0.98,1
2,0.0,0.32,0
3,1.0,0.92,1
4,1.0,0.98,1
...,...,...,...
1097,1.0,0.94,1
1098,1.0,0.89,1
1099,1.0,0.98,1
1100,1.0,0.91,1


## **Meta Classifier:**

In [149]:
from sklearn.linear_model import LogisticRegression

In [150]:
meta = LogisticRegression(fit_intercept= False,solver = 'lbfgs')

In [151]:
meta.fit(en[['clf_Tree','rf_Forest']],en['Attrition'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=False,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [152]:
en_test = pd.DataFrame()

In [153]:
en_test['clf_Tree'] = pd.DataFrame(clf.predict_proba(x_test))[1]
en_test['rf_Forest'] = pd.DataFrame(rf.predict_proba(x_test))[1]
col = en.columns
en_test['combined'] = meta.predict(en_test[['clf_Tree','rf_Forest']])

In [154]:
en_test = pd.concat([en_test,pd.DataFrame(y_test).reset_index(drop = True)],axis = 1)


In [155]:
en_test

,clf_Tree,rf_Forest,combined,Attrition
0,1.0,0.82,1,1
1,1.0,0.81,1,1
2,1.0,0.73,1,1
3,1.0,0.82,1,1
4,1.0,0.86,1,1
...,...,...,...,...
363,1.0,0.83,1,1
364,1.0,0.90,1,1
365,1.0,0.89,1,1
366,1.0,0.85,1,1


In [156]:
pd.crosstab(en_test['Attrition'],en_test['combined'])

combined,0,1
Attrition,,
0,20,37
1,40,271


In [157]:
round(accuracy_score(en_test['Attrition'],en_test['combined']),4)

0.7908

In [158]:
print(classification_report(en_test['Attrition'],en_test['combined']))

              precision    recall  f1-score   support

           0       0.33      0.35      0.34        57
           1       0.88      0.87      0.88       311

    accuracy                           0.79       368
   macro avg       0.61      0.61      0.61       368
weighted avg       0.80      0.79      0.79       368



## **Single Classifier:**

In [159]:
df

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,0,0,1102,0,1,2,0,2,1,94,3,2,0,4,0,5993,19479,8,0,11,3,1,0,8,0,1,6,4,0,5
1,49,1,1,279,1,8,1,0,3,0,61,2,2,1,2,1,5130,24907,1,1,23,4,4,1,10,3,3,10,7,1,7
2,37,0,0,1373,1,2,2,1,4,0,92,2,1,2,3,0,2090,2396,6,0,15,3,2,0,7,3,3,0,0,0,0
3,33,1,1,1392,1,3,4,0,4,1,56,3,1,1,3,1,2909,23159,1,0,11,3,3,0,8,3,3,8,7,3,0
4,27,1,0,591,1,2,1,2,1,0,40,3,1,2,2,1,3468,16632,9,1,12,3,4,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,1,1,884,1,23,2,2,3,0,41,4,2,2,4,1,2571,12290,4,1,17,3,3,1,17,3,3,5,2,0,3
1466,39,1,0,613,1,6,1,2,4,0,42,2,3,4,1,1,9991,21457,4,1,15,3,1,1,9,5,3,7,7,1,7
1467,27,1,0,155,1,4,3,0,2,0,87,4,2,3,2,1,6142,5174,1,0,20,4,2,1,6,0,3,6,2,0,3
1468,49,1,1,1023,0,2,3,2,4,0,63,2,2,0,2,1,5390,13243,2,1,14,3,4,0,17,3,2,9,6,0,8


In [160]:
df.Attrition.value_counts()/df.Attrition.count() # 1 - No, 0 - Yes

1    0.838776
0    0.161224
Name: Attrition, dtype: float64

In [161]:
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,BaggingClassifier

In [162]:
class_weight = {0:0.839,1:0.161} 

In [163]:
forest = RandomForestClassifier(class_weight=class_weight,n_estimators=100)

In [164]:
ada = AdaBoostClassifier(base_estimator=forest,n_estimators=100,learning_rate=0.5,random_state=42)

In [165]:
ada.fit(x_train,y_train.ravel())

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=RandomForestClassifier(bootstrap=True,
                                                         ccp_alpha=0.0,
                                                         class_weight={0: 0.839,
                                                                       1: 0.161},
                                                         criterion='gini',
                                                         max_depth=None,
                                                         max_features='auto',
                                                         max_leaf_nodes=None,
                                                         max_samples=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                              

In [166]:
print_score(ada,x_train,y_train,x_test,y_test,part = 'train')
print("\n***************************************************************\n")
print_score(ada,x_train,y_train,x_test,y_test,part = 'test')

Training Performance:


Accuracy:1.0000

Clasification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       180
           1       1.00      1.00      1.00       922

    accuracy                           1.00      1102
   macro avg       1.00      1.00      1.00      1102
weighted avg       1.00      1.00      1.00      1102


Confusion Matrix:
[[180   0]
 [  0 922]]

ROC_AUC_Score:1.0000


***************************************************************

Testing Performance:


Accuracy:0.8614

Clasification Report:
              precision    recall  f1-score   support

           0       0.75      0.16      0.26        57
           1       0.87      0.99      0.92       311

    accuracy                           0.86       368
   macro avg       0.81      0.57      0.59       368
weighted avg       0.85      0.86      0.82       368


Confusion Matrix:
[[  9  48]
 [  3 308]]

ROC_AUC_Score:0.5741



In [167]:
bag = BaggingClassifier(base_estimator=ada,n_estimators=50,n_jobs = -1,random_state=42)

In [168]:
bag.fit(x_train,y_train.ravel())

BaggingClassifier(base_estimator=AdaBoostClassifier(algorithm='SAMME.R',
                                                    base_estimator=RandomForestClassifier(bootstrap=True,
                                                                                          ccp_alpha=0.0,
                                                                                          class_weight={0: 0.839,
                                                                                                        1: 0.161},
                                                                                          criterion='gini',
                                                                                          max_depth=None,
                                                                                          max_features='auto',
                                                                                          max_leaf_nodes=None,
                                                   

In [169]:
print_score(bag,x_train,y_train,x_test,y_test,part = 'train')
print("\n***************************************************************\n")
print_score(bag,x_train,y_train,x_test,y_test,part = 'test')

Training Performance:


Accuracy:0.9564

Clasification Report:
              precision    recall  f1-score   support

           0       1.00      0.73      0.85       180
           1       0.95      1.00      0.97       922

    accuracy                           0.96      1102
   macro avg       0.98      0.87      0.91      1102
weighted avg       0.96      0.96      0.95      1102


Confusion Matrix:
[[132  48]
 [  0 922]]

ROC_AUC_Score:0.8667


***************************************************************

Testing Performance:


Accuracy:0.8560

Clasification Report:
              precision    recall  f1-score   support

           0       0.83      0.09      0.16        57
           1       0.86      1.00      0.92       311

    accuracy                           0.86       368
   macro avg       0.84      0.54      0.54       368
weighted avg       0.85      0.86      0.80       368


Confusion Matrix:
[[  5  52]
 [  1 310]]

ROC_AUC_Score:0.5423

